一、任务说明

训练vgg19整体表示+GRU模型，验证功能，选择相对合适的嵌入系统的模型参数文件。

二、实验数据


1、训练数据（参见getdata.py说明）

2、功能验证数据：

（1）随机选取的男女图片各一张，重命名为m/w_test1.jpg。
旨在观察能否区分男女性别差异，生成不一样的图片描述。

（2）随机生成的与图片预处理输入模型时大小一致的张量。
旨在观察模型应对随机数据的能力，能否生成贴切的图片描述，是否存在“编造”的情况。

三、实验环境


1、环境配置要求：4核CPU+8GB内存+1卡GPU

2、操作系统：Windows10+Linux

3、编写环境：jupyter lab

4、深度学习框架：依赖于深度学习框架PyTorch


四、所用的方法

1、使用vgg19整体表示+GRU模型训练实现有监督训练。（模型详细说明参见modelcr.py）

In [1]:
import torch

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.nn.utils.rnn import pack_padded_sequence
#引入评测指标，文本序列转文本的函数
from evaluate import evaluate_bleu_4, evaluate_rouge_l, fromtexttosentence
#引入词汇表生成函数，训练、验证、测试集划分函数
from getdata import words, mktrainvaltest
#引入模型定义
from modelcr import *

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /home/u2021213513/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u2021213513/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#test words:
train_path = './deepfashion-multimodal/train_captions.json'
all_words = words(train_path)
print(all_words)
print(len(all_words))

{'.': 1, 'is': 2, 'The': 3, 'a': 4, 'with': 5, 'fabric': 6, 'and': 7, 'patterns': 8, 'wears': 9, 'color': 10, 'cotton': 11, 'has': 12, 'her': 13, 'This': 14, 'on': 15, 'There': 16, 'shirt': 17, 'neckline': 18, 'tank': 19, 'pants': 20, 'pure': 21, 'solid': 22, 'are': 23, 'of': 24, 'an': 25, 'person': 26, 'ring': 27, 'long': 28, 'accessory': 29, ',': 30, 'wearing': 31, 'female': 32, 'lady': 33, 'wrist': 34, 'sleeves': 35, 'trousers': 36, 'graphic': 37, 'finger': 38, 'it': 39, 'clothing': 40, 'denim': 41, 'three-point': 42, 'shorts': 43, 'It': 44, 'top': 45, 'length': 46, 'the': 47, 'woman': 48, 'its': 49, 'in': 50, 'round': 51, 'sweater': 52, 'this': 53, 'crew': 54, 'his': 55, 'chiffon': 56, 'outer': 57, 'long-sleeve': 58, 'neck': 59, 'Her': 60, 'neckwear': 61, 'T-shirt': 62, 'hat': 63, 'short': 64, 'suspenders': 65, 'sleeveless': 66, 'short-sleeve': 67, 'upper': 68, 'lower': 69, 'v-shape': 70, 'knitting': 71, 'cut': 72, 'off': 73, 'no': 74, 'floral': 75, 'lapel': 76, 'head': 77, 'medium

In [16]:
#算力资源有限，为避免频繁be killed，放弃了加大batch并行加载数据
test_path = './deepfashion-multimodal/test_captions.json'
batch_size = 1
train_data_loader, val_data_loader, test_data_loader = mktrainvaltest(train_path, test_path, all_words, batch_size, workers = 0)

2、损失函数与超参数：


（1）损失函数：移除<pad>，计算交叉熵损失

（2）超参数设置：

①生成参数：max_len最大的生成长度为100，使用bleu_4指标评测时为每张图像生成的描述数captions_per_image为5，束搜索宽度beam_k为5。

②神经网络有关参数：image_code_dim编码器编码后的图像维度为4096，词嵌入维度word_dim为512，GRU解码器隐藏层hidden_size大小512，层数num_layers为1。

③学习率：encoder_learning_rate = 0.00001，decoder_learning_rate = 0.0005。

④训练epoch：num_epochs = 10。

⑤最大裁剪梯度：grad_clip = 5.0。

In [14]:
class PackedCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(PackedCrossEntropyLoss, self).__init__()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, predictions, targets, lengths):
        """
        参数：
            predictions：按文本长度排序过的预测结果
            targets：按文本长度排序过的文本描述
            lengths：文本长度
        """
        predictions = pack_padded_sequence(predictions, lengths, batch_first=True)[0]
        targets = pack_padded_sequence(targets, lengths, batch_first=True)[0]
        #print(predictions)
        #print(targets.add(-1))
        return self.loss_fn(predictions, targets.add(0))
def get_optimizer(model, encoder_learning_rate, decoder_learning_rate):
    return torch.optim.Adam([{"params": filter(lambda p: p.requires_grad, model.encoder.parameters()), 
                              "lr": encoder_learning_rate},
                             {"params": filter(lambda p: p.requires_grad, model.decoder.parameters()), 
                              "lr": decoder_learning_rate}])
# 设置模型超参数和辅助变量
max_len = 100
captions_per_image = 5
image_code_dim = 4096
word_dim = 512
hidden_size = 512
num_layers = 1
encoder_learning_rate = 0.00001
decoder_learning_rate = 0.0005
num_epochs = 10
grad_clip = 5.0

evaluate_step = 300 # 每隔多少步在验证集上测试一次
best_checkpoint_for_rouge = './model/MYMODL/best_rouge.ckpt' # 验证集上表现最优的模型的路径
best_checkpoint_for_bleu = './model/MYMODL/best_bleu.ckpt' # 验证集上表现最优的模型的路径
last_checkpoint = './model/MYMODL/last.ckpt' # 训练完成时的模型的路径
beam_k = 5
import os
# 设置GPU信息
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# 模型
# 随机初始化 或 载入已训练的模型
start_epoch = 0
checkpoint = last_checkpoint#None # 如果不为None，则利用该变量路径的模型继续训练
print(checkpoint)
if checkpoint is None:
    print("随机初始化")
    model = MYMODL(image_code_dim, all_words, word_dim, hidden_size, num_layers)
else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    model = checkpoint['model']

# 优化器
optimizer = get_optimizer(model, encoder_learning_rate, decoder_learning_rate)

# 将模型拷贝至GPU，并开启训练模式
model.to(device)
model.train()

./model/MYMODL/last.ckpt


MYMODL(
  (encoder): ImageEncoder(
    (image_encoder): VGG(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace=True)
        (14): Conv2d(256, 256, kernel_size=(3, 3), stride

五、实验结果

1、模型训练

In [17]:
import time
# 损失函数
loss_fn = PackedCrossEntropyLoss().to(device)

best_res_for_bleu = 0
best_res_for_rouge = 0
print("开始训练")
#不清空：a；清空：w
fw = open('log.txt', 'a')

for epoch in range(start_epoch, num_epochs):
    start_time = time.time()
    for i, (imgs, caps, caplens) in enumerate(train_data_loader):
        #接着训练
        if (i > 1800):
            optimizer.zero_grad()
            # 1. 读取数据至GPU
            imgs = imgs.to(device)
            caps = caps.to(device)
            caplens = caplens.to(device)

            # 2. 前馈计算
            predictions, sorted_captions, lengths, sorted_cap_indices = model(imgs, caps, caplens)
            # 3. 计算损失
            # captions从第2个词开始为targets
            loss = loss_fn(predictions, sorted_captions[:, 1:], lengths)

            loss.backward()
            # 梯度截断
            if grad_clip > 0:
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

            # 4. 更新参数
            optimizer.step()

            if (i+1) % 100 == 0:
                i_time = time.time() - start_time
                print('epoch %d, step %d: loss=%.2f, time=%.2f' % (epoch, i+1, loss.cpu(), i_time))
                fw.write('epoch %d, step %d: loss=%.2f \n' % (epoch, i+1, loss.cpu()))
                fw.flush()
                torch.save(model, f'./model/MYMODL/mymodl_{epoch}_{i}.pth')

            state = {
                    'epoch': epoch,
                    'step': i,
                    'model': model,
                    'optimizer': optimizer
                    }
            if (i+1) % evaluate_step == 0:
                #如果有必要，val可以从train里面分（交叉验证），否则用test
                bleu_score = evaluate_bleu_4(val_data_loader, model, captions_per_image, beam_k, max_len)
                rouge_score = evaluate_rouge_l(val_data_loader, model, beam_k, max_len)

                # 5. 选择模型
                if best_res_for_bleu < bleu_score:
                    best_res_for_bleu = bleu_score
                    torch.save(state, best_checkpoint_for_bleu)
                if best_res_for_rouge < rouge_score:
                    best_res_for_rouge = rouge_score
                    torch.save(state, best_checkpoint_for_rouge)
                torch.save(state, last_checkpoint)
                fw.write('Validation@epoch, %d, step, %d, BLEU-4=%.2f\n' % 
                      (epoch, i+1, bleu_score))
                fw.write('Validation@epoch, %d, step, %d, rouge-l=%.2f\n' % 
                      (epoch, i+1, rouge_score))
                fw.flush()
                print('Validation@epoch, %d, step, %d, BLEU-4=%.2f' % 
                      (epoch, i+1, bleu_score))
                print('Validation@epoch, %d, step, %d, rouge-l=%.2f' % 
                      (epoch, i+1, rouge_score))
fw.close()

开始训练
epoch 1, step 1000: loss=0.53, time=505.19
epoch 1, step 1100: loss=0.67, time=1032.10
epoch 1, step 1200: loss=0.88, time=1555.68
Validation@epoch, 1, step, 1200, BLEU-4=0.68
Validation@epoch, 1, step, 1200, rouge-l=0.17
epoch 1, step 1300: loss=0.69, time=4159.67
epoch 1, step 1400: loss=0.37, time=4675.91
epoch 1, step 1500: loss=0.64, time=5153.73
Validation@epoch, 1, step, 1500, BLEU-4=0.74
Validation@epoch, 1, step, 1500, rouge-l=0.17
epoch 1, step 1600: loss=0.65, time=7749.80
epoch 1, step 1700: loss=0.49, time=8225.30
epoch 1, step 1800: loss=0.46, time=8710.16
Validation@epoch, 1, step, 1800, BLEU-4=0.65
Validation@epoch, 1, step, 1800, rouge-l=0.18


KeyboardInterrupt: 

2、功能验证

In [18]:
from PIL import Image
from torchvision import transforms
from api import translate
def get_image(file_path):
    # 读取图像
    image = Image.open(file_path)
    image = image.convert('RGB')
    tx = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    #try2
    if tx is not None:
        image = tx(image).unsqueeze(0)
    return image
imgs = get_image('./sample_images/w_test1.jpg')
imgl = torch.randn(1, 3, 224, 224)
print(imgs.shape)
print(imgl.shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [19]:
model.eval()
e_textss = model.generate_by_beamsearch(imgs, 5, 102)
print(model.encoder(imgs))
print(e_textss)
e_textsl = model.generate_by_beamsearch(imgl, 5, 102)
print(model.encoder(imgl))
print(e_textsl)

tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
[[111, 60, 19, 45, 12, 74, 35, 30, 11, 6, 7, 21, 10, 8, 1, 3, 18, 24, 39, 2, 54, 1, 3, 69, 40, 2, 24, 42, 46, 1, 3, 6, 2, 41, 7, 39, 12, 22, 10, 8, 1, 112]]
tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ReluBackward1>)
[[111, 3, 68, 40, 12, 64, 35, 30, 11, 6, 7, 21, 10, 8, 1, 3, 18, 24, 39, 2, 54, 1, 3, 69, 40, 2, 24, 42, 46, 1, 3, 6, 2, 41, 7, 39, 12, 22, 10, 8, 1, 112]]


In [20]:
print(translate(fromtexttosentence(e_textss, all_words)))
print(translate(fromtexttosentence(e_textsl, all_words)))

<start>她的背心没有袖子，棉质面料和纯色图案。它的领口是圆领的。下面的衣服有三分长。面料是牛仔布，有纯色图案<end>
<start>上衣采用短袖、棉质面料和纯色图案。它的领口是圆领的。下面的衣服有三分长。面料是牛仔布，有纯色图案<end>


In [21]:
imgm = get_image('./sample_images/m_test1.jpg')

In [22]:
e_textsm = model.generate_by_beamsearch(imgm, 5, 102)
print(translate(fromtexttosentence(e_textsm, all_words)))

<start>她的背心没有袖子，棉质面料和纯色图案。它的领口是圆领的。下面的衣服有三分长。面料是牛仔布，有纯色图案。她的手腕上有一个配件<end>


六、实验结果分析


1、loss:

据上述实验结果和log.txt日志记录，loss在训练时波动减小，说明模型正在学习，且并未达到最优。

2、best_bleu_4（原理与实现参见evaluate.py）


（1）偶尔出现bleu为0的情况。

可能原因：该现象曾出现在一个epoch没训完下一个epoch继续从step1开始训练的时候，预测得到<start><end>，模型学习不到数据，可能是早期训练模型见的数据少的缘故，波动也就有了。

（2）bleu大小与模型生成描述与图片贴合度没有直接关联。

我们观察到，best_bleu有可能不如最后一次模型训练输出的效果好，生成的描述呈现高度重合性，相反，在输入图片数据随机性越大时，描述反而越丰富（虽然不一定贴切）。观察imageencoder之后的输出，认为可能是在编码器较简单、编码器解码器学习率相差不大的情况下，出现了一种“编解码无限拟合”的情况，让解码器认为输入的值是固定和相似的，编码器则认为要将图像编码为相似输出的，整体特征和归一化同时也加大了这种相似性，因而在小的epoch下，很难让这种差异更加明显，也就相对难以实现图片描述生成的多样性，因为我们的解码器倾向于生成更多它见过的词汇，这些词汇可能反映模型目前见过的数据的普遍特征（比如，我们可以推测，数据集中女性图片数据多于男性，且描述文本中对男性的描述往往、较少出现“a man”“a male”“he”“his”等词汇，因为模型总是认为一个图片是女性，而很少明确地说“他”）。

In [23]:
checkpoint = torch.load(best_checkpoint_for_bleu)
model = checkpoint['model']
model.eval()
e_textss = model.generate_by_beamsearch(imgs, 5, 102)
e_textsm = model.generate_by_beamsearch(imgm, 5, 102)
e_textsl = model.generate_by_beamsearch(imgl, 5, 102)
print(translate(fromtexttosentence(e_textss, all_words)))
print(translate(fromtexttosentence(e_textsm, all_words)))
print(translate(fromtexttosentence(e_textsl, all_words)))

<start>此人穿着纯色图案的无袖背心。这件背心是棉布的。它有一个吊带领口。这位女士穿的裤子有三分长。这条裤子采用棉布和图案。她的手腕上有一个配件<end>
<start>此人穿着纯色图案的无袖背心。这件背心是棉布的。它有一个吊带领口。这位女士穿的裤子有三分长。这条裤子采用棉布和图案。她的手腕上有一个配件<end>
<start>这位女士穿着一件纯色图案的无袖背心和一条长裤。这件背心是棉布的。这条裤子采用棉布和图案。这位女士戴着一枚戒指<end>


3、best_rouge_l（原理与实现参见evaluate.py）


（1）偶尔出现rouge为0的情况。

可能原因：与bleu类似。

（2）初步观察发现，rouge大小可能与模型生成描述多样性有关，rouge更小的情况下，主观观察模型对不同图片生成的描述更丰富。


In [24]:
checkpoint = torch.load(best_checkpoint_for_rouge)
model = checkpoint['model']
model.eval()
e_textss = model.generate_by_beamsearch(imgs, 5, 102)
e_textsm = model.generate_by_beamsearch(imgm, 5, 102)
e_textsl = model.generate_by_beamsearch(imgl, 5, 102)
print(translate(fromtexttosentence(e_textss, all_words)))
print(translate(fromtexttosentence(e_textsm, all_words)))
print(translate(fromtexttosentence(e_textsl, all_words)))

<start>上衣采用长袖、棉质面料和纯色图案。它有一个圆领。下面的衣服有三分长。面料是牛仔布，有纯色的花纹。这个人戴着戒指<end>
<start>上衣采用长袖、棉质面料和纯色图案。它有一个圆领。下面的衣服有三分长。面料是牛仔布，有纯色的花纹。这个人戴着戒指<end>
<start>上衣采用短袖、棉质面料和纯色图案。它有一个圆领。下面的衣服有三分长。面料是牛仔布，有纯色的花纹。这个人戴着戒指<end>


在实验中还观察到，rouge与bleu相比明显更小，波动更弱，说明该评测标准在本实验下相对更严格也更难优化，参考价值或许不如bleu，因而后续模型可以考虑采用bleu来进行评测，或者尝试引入其他的指标如准确率等。

In [25]:
i = 99
while (i < 1800):
    print(i)
    try:
        model = torch.load(f'./model/MYMODL/mymodl_0_{i}.pth')
    except:
        model = torch.load(f'./model/MYMODL/mymodl_1_{i}.pth')
    model.eval()
    e_textss = model.generate_by_beamsearch(imgs, 5, 102)
    e_textsm = model.generate_by_beamsearch(imgm, 5, 102)
    e_textsl = model.generate_by_beamsearch(imgl, 5, 102)
    print(translate(fromtexttosentence(e_textss, all_words)))
    print(translate(fromtexttosentence(e_textsm, all_words)))
    print(translate(fromtexttosentence(e_textsl, all_words)))
    i += 100

99
<start>戴戒指的人手腕上戴着戒指。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。有
<start>戴戒指的人手腕上戴着戒指。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件。她的手腕上有一个配件<end>
<start>这件衬衫有袖子、袖子、袖子，棉质面料和纯色图案。这只雌性的手指上戴着一枚戒指。这只雌性的手指上戴着一枚戒指。她的手指上戴着一枚戒指。她的手指上戴着一枚戒指。她的手指上戴着一枚戒指。她的手指上戴着一枚戒指。她的手指上戴着一枚戒指。她的手指上戴着一枚戒指。她的手指上戴着一枚戒指。
199
<start>这件衬衫穿着一件纯色图案的长袖背心和一条长裤。这件背心采用纯棉面料，纯色图案。它的领口是圆领的。这条裤子是纯棉的，有纯色的花纹。它的领口是圆领的。这条裤子是纯棉的，有纯色的花纹。它的领口是圆领的。这条裤子是纯棉的，有纯色的花纹。它的领口是圆领的。这条裤子是纯棉的，有纯色的花纹。
<start>这位女士穿着一件纯色图案的长袖背心和一条长裤。这件背心采用纯棉面料，纯色图案。它的领口是圆领的。这条裤子是纯棉的，有纯色的花纹。她的手腕上有一个配件<end>
<start>衬衫穿着长袖衬衫，纯棉面料，纯色图案，纯色图案和纯色图案。这件衬衫是纯棉的，有纯色的花纹。这件衬衫是纯棉的，有纯色的花纹。这件衬衫是纯棉的，有纯色的花纹。这件衬衫是纯棉的，有纯色的花纹。这件衬衫是纯棉的，有纯色的花纹。这件衬衫是纯棉的，有纯色的花纹。这件衬衫采用纯棉面料，有纯色图案
299
<start>这名女性穿着纯色图案的背心。背心采用纯棉面料和纯色图案。雌性戴着戒指<end>
<start>这名女性穿着纯色图案的背心。这件背心是纯棉的，有纯色的花纹。雌性戴着戒指<end>
<start>上衣采用长袖、棉质面料和纯色图案。这件衬衫是纯棉的，有纯色的花纹。女的穿一条长裤。这条裤子是纯棉的，有纯色的花纹。女的穿一条长裤。这条裤子是纯棉的，有纯色的花纹。雌性戴着戒指<end>
399
<start>这名女性穿着印有图案的背心。背心是棉布的。它有一个圆形的领口。这条裤子是纯棉的，有

In [26]:
#结合rouge和主观认知，选择模型加载保存作为系统嵌入的模型
torch.load('./model/MYMODL/mymodl_0_799.pth')
torch.save(model, f'./model/MYMODL/mymodl.pth')

七、总结


1、通过本实验，加深了对编码器解码器结构的理解，完成了从image到text的生成尝试，为多模态应用和进一步打开深度学习的世界打下了厚实的实践基础。

2、本模型的工作量在于手写实现vgg19+gru的编解码结构，深入理解了图片编码数据嵌入序列模型的机制，通过简单的参数设置和调优训练，基本实现了从服装图片生成文本描述的任务。

3、创新点在于：对比两种评测指标在模型描述生成上的表现，同时引入api翻译生成中文描述以直观感知描述准确度，评测和调优相对客观和完善。

4、局限在于：

（1）训练数据词汇量少、丰富性不足；

（2）算力限制了模型超参数的调优，因为每重新加载模型，就会产生大量未释放的内存，造成内存溢出的隐患，而若想要消除隐患，则需每次重启内核，重新加载词典和Dataloader，时间开销不可估量；

（3）内存和算力限制了模型训练，由于单步batchsize为1、workers为0才能保证内存足够适应后续训练，数据加载速度缓慢，加之vgg19模型本身计算复杂度高，平均每训练100个数据耗时近10分钟，按训练集大小9000余、验证集大小1000余来算，每训练一个epoch需要至少15h，同时每验证一次评估两个指标大约需20分钟，即使900步评估一次也需要6个小时，即训练一个完整的epoch需要电脑在虚拟环境下不眠不休工作一整天，极大地限制了模型的训练；

（4）vgg19作为imageencoder提取整体特征对于背景纯白的模特服装图片来讲，提出的特征差异较难满足本实验要求，应选择更细粒度的模型，同时也要兼顾计算复杂度，比如固定预训练模型的某些层不动，只做微调（因为9000个数据对于本例来说数据量已经算相对充足的了），以此来实现进一步优化；

（5）有研究表明，gru作为解码器训练更快但LSTM在数据量足够的情况下表征能力更强，本例后续也可以使用LSTM实现解码器以此进一步探究在该图像描述生成任务下，针对同一编码器，两者对本实验的适应度。
